In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import os
from glob import glob
import imageio

def draw_flow_arrows(image, flow, step=10, color=(0, 255, 0), thickness=2):
    """Draw arrows representing optical flow on an image."""
    h, w = image.shape[:2]
    for y in range(0, h, step):
        for x in range(0, w, step):
            fx, fy = flow[y, x]
            end_point = (int(x + fx), int(y + fy))
            magnitude = np.hypot(fx, fy)
            if magnitude > 1.0:
                cv2.arrowedLine(image, (x, y), end_point, color, thickness, tipLength=0.4)
    return image

def flow_to_hsv(flow):
    """Convert flow to HSV image."""
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=np.uint8)
    hsv[..., 0] = ang * 180 / np.pi / 2  # Hue
    hsv[..., 1] = 255  # Saturation
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)  # Value
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def save_gif_from_frames(frame_paths, gif_path, duration=0.1):
    frames = [cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB) for p in frame_paths]
    imageio.mimsave(gif_path, frames, duration=duration)

def visualize_optical_flow_with_arrows(frames_folder, output_dir='output_flow_arrows', beta=0.7):
    os.makedirs(output_dir, exist_ok=True)
    dirs = {
        "flow_arrows": "arrow",
        "flow_arrows_raw": "raw_arrow",
        "flow_hsv": "hsv",
        "flow_hsv_raw": "raw_hsv"
    }
    for d in dirs:
        os.makedirs(os.path.join(output_dir, d), exist_ok=True)

    frame_paths = sorted(glob(os.path.join(frames_folder, '*')))
    if len(frame_paths) < 2:
        print("Need at least 2 frames.")
        return

    prev_frame = cv2.imread(frame_paths[0])
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    acc_flow = None

    for i in range(1, len(frame_paths)):
        curr_frame = cv2.imread(frame_paths[i])
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, curr_gray, None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )

        # Save raw flow arrow
        raw_arrow = curr_frame.copy()
        draw_flow_arrows(raw_arrow, flow, step=10, color=(0, 0, 255), thickness=2)
        cv2.imwrite(os.path.join(output_dir, "flow_arrows_raw", f"{dirs['flow_arrows_raw']}_{i:02d}.png"), raw_arrow)

        # Save raw HSV
        raw_hsv = flow_to_hsv(flow)
        cv2.imwrite(os.path.join(output_dir, "flow_hsv_raw", f"{dirs['flow_hsv_raw']}_{i:02d}.png"), raw_hsv)

        # Update accumulated flow
        if acc_flow is None:
            acc_flow = flow
        else:
            acc_flow = beta * flow + (1 - beta) * acc_flow

        # Save accumulated arrow
        acc_arrow = curr_frame.copy()
        draw_flow_arrows(acc_arrow, acc_flow, step=10, color=(0, 255, 0), thickness=2)
        cv2.imwrite(os.path.join(output_dir, "flow_arrows", f"{dirs['flow_arrows']}_{i:02d}.png"), acc_arrow)

        # Save accumulated HSV
        acc_hsv = flow_to_hsv(acc_flow)
        cv2.imwrite(os.path.join(output_dir, "flow_hsv", f"{dirs['flow_hsv']}_{i:02d}.png"), acc_hsv)

        prev_gray = curr_gray

    print("Saved visualizations. Creating GIFs...")

    # Generate GIFs
    for subdir, prefix in dirs.items():
        images = sorted(glob(os.path.join(output_dir, subdir, f"{prefix}_*.png")))
        gif_path = os.path.join(output_dir, f"{prefix}.gif")
        save_gif_from_frames(images, gif_path, duration=0.1)
        print(f"Saved {prefix}.gif to {gif_path}")

# Example usage
visualize_optical_flow_with_arrows(
    "/content/drive/MyDrive/optical flow clip/clip 2 fight",
    output_dir="/content/drive/MyDrive/optical flow clip/flow_output_arrows",
    beta=0.7
)


Saved visualizations. Creating GIFs...
Saved arrow.gif to /content/drive/MyDrive/optical flow clip/flow_output_arrows/arrow.gif
Saved raw_arrow.gif to /content/drive/MyDrive/optical flow clip/flow_output_arrows/raw_arrow.gif
Saved hsv.gif to /content/drive/MyDrive/optical flow clip/flow_output_arrows/hsv.gif
Saved raw_hsv.gif to /content/drive/MyDrive/optical flow clip/flow_output_arrows/raw_hsv.gif
